In [32]:
import json
import pandas as pd
import boto3
from sagemaker import get_execution_role
import tarfile
from pprint import pprint

In [7]:
role = get_execution_role()
bucket='knarig-comprehend-bucket'
data_key = 'sagemaker/comprehend/output/347550782223-CLN-2fbb86de8af99a092e036b6626b568a5/output/output.tar.gz'
data_location = 's3://{}/{}'.format(bucket, data_key)

s3 = boto3.client('s3')
s3.download_file(bucket, data_key, 'output.tar.gz')

In [33]:
role = get_execution_role()
bucket='knarig-comprehend-bucket'
data_key = 'output/347550782223-CLN-31e061a9a4db6756ed48f2f3838e1333/output/output.tar.gz'
data_location = 's3://{}/{}'.format(bucket, data_key)

s3 = boto3.client('s3')
s3.download_file(bucket, data_key, 'output.tar.gz')

In [34]:
tar = tarfile.open('output.tar.gz', "r:gz")
tar.extractall()

In [35]:
with open('predictions.jsonl', 'r') as f:
    f = f.readlines()

In [36]:
pprint(f)

['{"File": "test_s3.csv", "Line": "0", "Classes": [{"Name": "business", '
 '"Score": 0.6823}, {"Name": "entertainment", "Score": 0.1373}, {"Name": '
 '"sport", "Score": 0.0718}]}\n',
 '{"File": "test_s3.csv", "Line": "1", "Classes": [{"Name": "business", '
 '"Score": 0.8779}, {"Name": "entertainment", "Score": 0.0441}, {"Name": '
 '"tech", "Score": 0.0363}]}\n',
 '{"File": "test_s3.csv", "Line": "2", "Classes": [{"Name": "business", '
 '"Score": 0.864}, {"Name": "politics", "Score": 0.0945}, {"Name": "sport", '
 '"Score": 0.021}]}\n',
 '{"File": "test_s3.csv", "Line": "3", "Classes": [{"Name": "business", '
 '"Score": 0.9506}, {"Name": "entertainment", "Score": 0.0195}, {"Name": '
 '"politics", "Score": 0.0138}]}\n',
 '{"File": "test_s3.csv", "Line": "4", "Classes": [{"Name": "business", '
 '"Score": 0.9393}, {"Name": "politics", "Score": 0.0304}, {"Name": "tech", '
 '"Score": 0.0185}]}\n',
 '{"File": "test_s3.csv", "Line": "5", "Classes": [{"Name": "business", '
 '"Score": 0.9225}, {"

In [37]:
# predicted label list
predictedLabels = []
# looping through json lines
for file in f:
    # casting str json to json
    json_data = json.loads(file)["Classes"]
    # fetching class with maximum score
    predictedLabels.append(json_data[0]['Name'])

In [38]:
# reading test document
df_test = pd.read_csv("test.csv", header=None)
# assigning header
df_test.columns = ["Document"]

In [39]:
# creating new column and mapping/assigning label
df_test["PredictedLabel"] = predictedLabels

In [40]:
# readinf test truth file
df_truth = pd.read_csv("test_truth.csv", header=None)
# assigning header
df_truth.columns = ["TruthLabel", "Document"]

In [41]:
# merging both dataframe on Document column
mergeDf = pd.merge(df_test, df_truth, on=["Document"], how="inner")

In [42]:
# checking/comparing two columns
mergeDf.PredictedLabel == mergeDf.TruthLabel

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
30    True
31    True
32    True
33    True
34    True
35    True
36    True
37    True
38    True
39    True
40    True
41    True
42    True
43    True
44    True
45    True
46    True
47    True
48    True
49    True
dtype: bool

In [43]:
mergeDf[mergeDf.PredictedLabel == mergeDf.TruthLabel].count()


Document          50
PredictedLabel    50
TruthLabel        50
dtype: int64